### Basic zip archive data access and usage
For simplicity, the following example loads only 200 sweeps directly from the zipfile into pandas DataFrame objects. These operations can be significantly accelerated using dask instead (see `dask_processing.ipynb`).

In order to run this yourself, you'll need to download a zip file archive, and adjust `data_path` accordingly.

In [1]:
import sea_ingest
import xarray as xr
import pandas as pd
import numpy as np
from labbench import stopwatch


def sea_edge_data_to_xarray(
    df: pd.DataFrame, coord_name: str | None = None, attrs: dict = {}
):
    (_, datetimes), *name_levels = zip(df.index.names, df.index.levels)
    index_coords = {name: np.array(level) for name, level in name_levels}

    acq_count = int(df.shape[0] / np.prod([len(l) for l in index_coords.values()]))

    coords = {
        'start_time': np.empty(acq_count),  # placeholder
        **index_coords,
        (coord_name or df.columns.name): df.columns.values,
    }

    shape = tuple([len(v) for v in coords.values()])
    values = df.values.reshape(shape)
    datetimes = datetimes.values.reshape(shape[:2])
    coords['start_time'] = datetimes[:, 0]

    return xr.DataArray(
        values, coords=xr.Coordinates(coords), attrs={'label': df.columns.name, **attrs}
    )


def sea_edge_data_to_timestamps(df):
    (_, datetimes), *name_levels = zip(df.index.names, df.index.levels)
    index_coords = {name: np.array(level) for name, level in name_levels}

    acq_count = int(df.shape[0] / np.prod([len(l) for l in index_coords.values()]))

    time_coords = {
        'start_time': np.empty(acq_count),  # placeholder
        **index_coords,
    }

    time_shape = tuple([len(v) for v in time_coords.values()])

    datetimes = datetimes.values.reshape(time_shape)
    time_coords['start_time'] = datetimes[:, *([0] * (datetimes.ndim - 1))]

    return xr.DataArray(
        df.index.get_level_values('datetime').values.reshape(time_shape),
        coords=xr.Coordinates(time_coords),
    )


def pandas_dicts_to_dataset(dfs):
    return xr.Dataset(
        {
            'timestamp': sea_edge_data_to_timestamps(dfs['apd']),
            'pfp': sea_edge_data_to_xarray(
                dfs['pfp'],
                coord_name='cyclic_lag',
                attrs={'units': 'dBm/10 MHz'},
            ),
            'apd': sea_edge_data_to_xarray(
                dfs['apd'],
                coord_name='instantaneous_power',
                attrs={'units': '#'},
            ),
            'psd': sea_edge_data_to_xarray(
                dfs['psd'],
                coord_name='baseband_frequency',
                attrs={'units': 'dBm/? MHz'},
            ),
            'pvt': sea_edge_data_to_xarray(
                dfs['pvt'],
                coord_name='time_elapsed',
                attrs={'units': 'dBm/10 MHz'},
            ),
        }
    )


# %timeit -n1 -r1 pandas_dicts_to_dataset(dfs)
#

In [5]:
import dask
import typing
import zarr

partition_size = 200
data_path = 'data/HU_AllData-2023-06-01.zip'
store = zarr.DirectoryStore('zarr_output')


def init_store():
    dfs = sea_ingest.read_seamf_zipfile(
        data_path, allow=partition_size, tz='America/Denver', localize=False
    )
    pandas_dicts_to_dataset(dfs).to_zarr(store, mode='w')


def save_zarr(dfs: typing.Dict[str, pd.DataFrame]):
    ds = pandas_dicts_to_dataset(dfs)
    try:
        ds.to_zarr(store, mode='a', append_dim='start_time')
    except ValueError as ex:
        print(ex)


with dask.config.set(scheduler='processes', potato=74), stopwatch('setup'):
    init_store()

    delayed = sea_ingest.read_seamf_zipfile_as_delayed(
        data_path,
        partition_func=save_zarr,
        partition_size=partition_size,
        tz='America/Denver',
        localize=False,
    )

    dask.compute(delayed, num_workers=20)

found conflicting lengths for dimension start_time (800 != 600)
found conflicting lengths for dimension start_time (600 != 800)
found conflicting lengths for dimension start_time (800 != 1000)
found conflicting lengths for dimension start_time (800 != 1000)
found conflicting lengths for dimension start_time (1400 != 1000)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflicting lengths for dimension start_time (1200 != 1400)
found conflictin

 INFO   2024-05-10 13:23:16.192 • labbench: setup 17.772 s elapsed


In [8]:
ds = xr.load_dataset('zarr_output', engine='zarr')

ValueError: conflicting sizes for dimension 'start_time': length 1200 on 'psd' and length 1400 on {'start_time': 'apd', 'frequency': 'apd', 'instantaneous_power': 'apd', 'baseband_frequency': 'baseband_frequency', 'capture_statistic': 'capture_statistic', 'cyclic_lag': 'cyclic_lag', 'detector': 'detector'}

In [6]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [ ]:
import pickle
from pathlib import Path

# %timeit -n1 -r1 ds.to_netcdf('test.thingy', format="NETCDF4")

test_file = Path('test.p')
with open(test_file, 'wb') as fd:
    %timeit -n1 -r1 pickle.dump(ds, fd)

with open(test_file, 'rb') as fd:
    %timeit -n1 -r1 pickle.load(fd)

print(f'file size: {test_file.stat().st_size/1e6:0.1f} MB')

8.37 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
4.09 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
file size: 21.1 MB


In [ ]:
# import zarr
# compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)

test_file = Path('test.zarr.tar')
store = ds.to_zarr(test_file, mode='w')

%timeit -n1 -r1 ds.to_zarr(test_file, append_dim='acquisition')
%timeit -n1 -r1 xr.open_dataset(test_file, engine='zarr')

d = xr.open_dataset(
    test_file,
    engine='zarr',
)

print(f'file size: {test_file.stat().st_size/1e6:0.1f} MB')

62.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.74 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
file size: 0.0 MB


### Returned dictionary structure
The data are returned as a dictionary of `pd.DataFrame`, named by data product or metadata type.

### DataFrame structure
The data products are arranged as tables.
* The trace axis (time elapsed, FFT bin frequency, etc.) is given by the `column` attribute
* The trace index (timestamp, RF center frequency, and any trace specificiations like the detector) are arranged as levels of a multilevel index.

Some advantages to arranging the table this way:
* All data values (below, `dfs['pfp'].values`) are the same kind of quantity, in this case dBm/10 MHz. (TODO: attach units with pint :))
    - This means that operations like `10**(dfs['pfp']/10)` do not apply to the index
* We can use _any_ of the indexing metadata fields to query subsets of the data quickly

In [ ]:
dfs['pfp']

Frame time elapsed (s)                                                     0.000000  \
datetime                         frequency    capture_statistic detector              
2023-05-12 13:16:57.538000-06:00 3.545000e+09 min               rms      -100.12500   
                                              max               rms       -98.93750   
                                              mean              rms       -99.56250   
                                              min               peak      -93.56250   
                                              max               peak      -87.43750   
...                                                                             ...   
2023-05-12 14:39:12.140000-06:00 3.705000e+09 max               rms       -51.37500   
                                              mean              rms       -52.03125   
                                              min               peak      -45.81250   
                                              max               peak      -40.81250   
                                              mean              peak      -43.56250   

Frame time elapsed (s)                                                     0.000018  \
datetime                         frequency    capture_statistic detector              
2023-05-12 13:16:57.538000-06:00 3.545000e+09 min               rms      -100.31250   
                                              max               rms       -99.12500   
                                              mean              rms       -99.56250   
                                              min               peak      -93.25000   
                                              max               peak      -88.56250   
...                                                                             ...   
2023-05-12 14:39:12.140000-06:00 3.705000e+09 max               rms       -51.28125   
                                              mean              rms       -52.00000   
                                              min               peak      -45.68750   
                                              max               peak      -41.56250   
                                              mean              peak      -43.75000   

Frame time elapsed (s)                                                     0.000036  \
datetime                         frequency    capture_statistic detector              
2023-05-12 13:16:57.538000-06:00 3.545000e+09 min               rms      -100.12500   
                                              max               rms       -99.00000   
                                              mean              rms       -99.56250   
                                              min               peak      -93.50000   
                                              max               peak      -88.37500   
...                                                                             ...   
2023-05-12 14:39:12.140000-06:00 3.705000e+09 max               rms       -51.28125   
                                              mean              rms       -52.03125   
                                              min               peak      -45.34375   
                                              max               peak      -41.68750   
                                              mean              peak      -43.75000   

Frame time elapsed (s)                                                     0.000054  \
datetime                         frequency    capture_statistic detector              
2023-05-12 13:16:57.538000-06:00 3.545000e+09 min               rms      -100.12500   
                                              max               rms       -98.93750   
                                              mean              rms       -99.50000   
                                              min               peak      -93.18750   
                                              max               peak      -88.4375

### Quick indexing tutorial

You can access each index level using the index value. One way is with the `.loc` accessor, specifying `axis=0` to indicate that all slices are applied to the index (otherwise the 2nd field applies to columns). For example:

In [ ]:
dfs['pfp'].loc(axis=0)[:'2023-09-17 12:00', 3.555e9, 'max', 'rms']

,,,Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,frequency,capture_statistic,detector,,,,,,,,,,,,,,,,,,,,,
2023-05-12 13:16:57.538000-06:00,3.555000e+09,max,rms,-72.6875,-73.4375,-74.3125,-74.0625,-75.9375,-75.2500,-76.1875,-77.2500,-77.8750,-76.9375,...,-77.6250,-78.8750,-80.3125,-80.1875,-78.5625,-73.5625,-74.5625,-74.7500,-73.1875,-72.6250
2023-05-12 13:17:12.817000-06:00,3.555000e+09,max,rms,-79.5000,-76.0000,-76.1875,-76.3750,-76.3125,-75.9375,-77.4375,-77.6875,-78.1875,-78.0000,...,-73.5625,-75.1250,-73.8125,-75.3750,-77.6875,-77.4375,-79.2500,-80.2500,-78.6875,-79.5000
2023-05-12 13:17:23.167000-06:00,3.555000e+09,max,rms,-75.9375,-77.0000,-77.5625,-78.3750,-78.4375,-77.3750,-77.9375,-76.5000,-75.8750,-75.9375,...,-78.7500,-80.5625,-79.9375,-79.0625,-79.2500,-76.6250,-76.3125,-75.1250,-76.0625,-74.4375
2023-05-12 13:17:38.457000-06:00,3.555000e+09,max,rms,-72.9375,-73.3125,-73.5625,-74.6250,-76.4375,-76.3125,-77.2500,-77.8125,-79.1875,-78.8750,...,-77.5625,-78.9375,-78.7500,-79.3125,-75.5000,-75.0000,-74.2500,-74.3750,-75.1250,-73.3125
2023-05-12 13:17:53.516000-06:00,3.555000e+09,max,rms,-77.4375,-77.0625,-78.0000,-74.9375,-75.0000,-73.5000,-74.8750,-74.8750,-74.2500,-73.1250,...,-77.9375,-76.2500,-75.6875,-74.9375,-74.8125,-75.1250,-76.6875,-77.4375,-77.6250,-77.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 14:36:14.264000-06:00,3.555000e+09,max,rms,-76.3750,-76.8750,-75.6250,-75.2500,-76.4375,-75.4375,-78.5000,-78.7500,-80.5625,-84.7500,...,-73.6875,-72.2500,-73.9375,-77.0625,-77.6875,-78.5625,-79.3750,-87.0625,-85.1875,-83.7500
2023-05-12 14:38:15.589000-06:00,3.555000e+09,max,rms,-75.0000,-75.1250,-73.5000,-74.3125,-74.2500,-74.4375,-73.5000,-73.3750,-76.0000,-77.1250,...,-75.3125,-74.6250,-75.2500,-77.0000,-79.5000,-79.0625,-81.6875,-84.8750,-84.7500,-74.8125
2023-05-12 14:38:31.508000-06:00,3.555000e+09,max,rms,-99.3750,-99.3750,-99.3125,-99.4375,-99.3750,-99.2500,-99.3750,-99.3125,-99.3125,-99.4375,...,-99.3750,-99.2500,-99.3125,-99.3750,-99.2500,-99.3750,-99.3750,-99.2500,-99.4375,-99.1250


In many cases, we'd like to index a single value in a given level, especially for categorical data like the string-referred `capture_statistic` and `detector` fields. For this, pandas provides the `.xs` accessor.

In [ ]:
dfs['pfp'].xs(key=3.555e9, level='frequency')

Frame time elapsed (s)                                       0.000000  \
datetime                         capture_statistic detector             
2023-05-12 13:16:57.538000-06:00 min               rms       -79.6875   
                                 max               rms       -72.6875   
                                 mean              rms       -76.8125   
                                 min               peak      -70.8750   
                                 max               peak      -64.6875   
...                                                               ...   
2023-05-12 14:38:57.093000-06:00 max               rms       -76.1250   
                                 mean              rms       -81.8750   
                                 min               peak      -79.1875   
                                 max               peak      -69.5000   
                                 mean              peak      -73.8750   

Frame time elapsed (s)                                       0.000018  \
datetime                         capture_statistic detector             
2023-05-12 13:16:57.538000-06:00 min               rms       -79.9375   
                                 max               rms       -73.4375   
                                 mean              rms       -77.3750   
                                 min               peak      -71.5000   
                                 max               peak      -63.0625   
...                                                               ...   
2023-05-12 14:38:57.093000-06:00 max               rms       -75.3125   
                                 mean              rms       -82.7500   
                                 min               peak      -78.3750   
                                 max               peak      -68.0625   
                                 mean              peak      -74.2500   

Frame time elapsed (s)                                       0.000036  \
datetime                         capture_statistic detector             
2023-05-12 13:16:57.538000-06:00 min               rms       -79.8125   
                                 max               rms       -74.3125   
                                 mean              rms       -77.2500   
                                 min               peak      -71.3750   
                                 max               peak      -64.8750   
...                                                               ...   
2023-05-12 14:38:57.093000-06:00 max               rms       -79.1250   
                                 mean              rms       -84.0625   
                                 min               peak      -78.9375   
                                 max               peak      -69.5625   
                                 mean              peak      -74.6875   

Frame time elapsed (s)                                       0.000054  \
datetime                         capture_statistic detector             
2023-05-12 13:16:57.538000-06:00 min               rms       -79.4375   
                                 max               rms       -74.0625   
                                 mean              rms       -77.3750   
                                 min               peak      -71.5625   
                                 max               peak      -64.3125   
...                                                               ...   
2023-05-12 14:38:57.093000-06:00 max               rms       -79.8125   
                                 mean              rms       -85.2500   
                                 min               peak      -79.6875   
                                 max               peak      -70.1250   
                                 mean              peak      -75.3750   

Frame time elapsed (s)                                       0.000071  \
datetime                         capture_statistic detector             
2023-05-12 13:16:57.538000-06:00 min               rms

For flexible queries to single index values of a specified data product and multiple index levels at a time, you can also use `seamf.trace`. In the following example, notice that this drops the selected levels from the index. 

In [ ]:
sea_ingest.trace(
    dfs, 'pfp', frequency=3.555e9, capture_statistic='max', detector='rms'
).loc[:'2023-09-20 20:55']

Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,,,,,,,,,,,,,,,,,,,,,
2023-05-12 13:16:57.538000-06:00,-72.6875,-73.4375,-74.3125,-74.0625,-75.9375,-75.2500,-76.1875,-77.2500,-77.8750,-76.9375,...,-77.6250,-78.8750,-80.3125,-80.1875,-78.5625,-73.5625,-74.5625,-74.7500,-73.1875,-72.6250
2023-05-12 13:17:12.817000-06:00,-79.5000,-76.0000,-76.1875,-76.3750,-76.3125,-75.9375,-77.4375,-77.6875,-78.1875,-78.0000,...,-73.5625,-75.1250,-73.8125,-75.3750,-77.6875,-77.4375,-79.2500,-80.2500,-78.6875,-79.5000
2023-05-12 13:17:23.167000-06:00,-75.9375,-77.0000,-77.5625,-78.3750,-78.4375,-77.3750,-77.9375,-76.5000,-75.8750,-75.9375,...,-78.7500,-80.5625,-79.9375,-79.0625,-79.2500,-76.6250,-76.3125,-75.1250,-76.0625,-74.4375
2023-05-12 13:17:38.457000-06:00,-72.9375,-73.3125,-73.5625,-74.6250,-76.4375,-76.3125,-77.2500,-77.8125,-79.1875,-78.8750,...,-77.5625,-78.9375,-78.7500,-79.3125,-75.5000,-75.0000,-74.2500,-74.3750,-75.1250,-73.3125
2023-05-12 13:17:53.516000-06:00,-77.4375,-77.0625,-78.0000,-74.9375,-75.0000,-73.5000,-74.8750,-74.8750,-74.2500,-73.1250,...,-77.9375,-76.2500,-75.6875,-74.9375,-74.8125,-75.1250,-76.6875,-77.4375,-77.6250,-77.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 14:36:14.264000-06:00,-76.3750,-76.8750,-75.6250,-75.2500,-76.4375,-75.4375,-78.5000,-78.7500,-80.5625,-84.7500,...,-73.6875,-72.2500,-73.9375,-77.0625,-77.6875,-78.5625,-79.3750,-87.0625,-85.1875,-83.7500
2023-05-12 14:38:15.589000-06:00,-75.0000,-75.1250,-73.5000,-74.3125,-74.2500,-74.4375,-73.5000,-73.3750,-76.0000,-77.1250,...,-75.3125,-74.6250,-75.2500,-77.0000,-79.5000,-79.0625,-81.6875,-84.8750,-84.7500,-74.8125
2023-05-12 14:38:31.508000-06:00,-99.3750,-99.3750,-99.3125,-99.4375,-99.3750,-99.2500,-99.3750,-99.3125,-99.3125,-99.4375,...,-99.3750,-99.2500,-99.3125,-99.3750,-99.2500,-99.3750,-99.3750,-99.2500,-99.4375,-99.1250
